In [1]:
import requests
import json
import csv
import configparser

## Define a function to ask for the environment and match to config-template values.

In [2]:
def singleenvironment(configNames):
    environmentname = input("Which environment are we uploading reserves to? ")
    if environmentname in configNames:
        return environmentname
    else:
        sys.exit("can't recognize environment name!")

## Read in the environments you might be using from the config-template.ini file

In [3]:
envConfig = configparser.ConfigParser()
envConfig.read('config-template.ini')
print(envConfig.sections())

['snapshot']


In [4]:
uploadenvironment = singleenvironment(envConfig.sections())

Which environment are we uploading reserves to?  snapshot


## Define the header values to be used in making your API request to FOLIO.

In [5]:
uploadurl = envConfig[uploadenvironment]['okapi_url']

fetchHeaders = {
    'x-okapi-tenant': envConfig[uploadenvironment]['tenant_id'],
    'x-okapi-token': envConfig[uploadenvironment]['password']
}
postHeaders = {
    'x-okapi-tenant': envConfig[uploadenvironment]['tenant_id'],
    'x-okapi-token': envConfig[uploadenvironment]['password'],
    'Content-Type': 'application/json'
}

## Create a reserve

The FOLIO API to create a reserve is

POST /coursereserves/reserves

with the required fields of the course listing ID and the barcode. 

The goal is to take an input file with a list of barcodes, one on each line, and construct a JSON object for each barcode like this:

{
    "courseListingId": *"yourcourselistingid"*,
    "copiedItem": {
        "barcode": *"itembarcode"*
        }
}

and then call a POST /coursereserves/reserves with the JSON for each item until the list is finished.


**Step 1**: Create the URL for the reserve API call.

In [6]:
uploadReserveUrl = '{}{}'.format(uploadurl, "/coursereserves/reserves")

**Step 2**: Ask for the course listing ID.

In [7]:
courseListingIdForLoad = input("Please provide the UUID for the course listing we are adding items to ")

Please provide the UUID for the course listing we are adding items to  cef52efb-b3fd-4450-9960-1745026a99d1


**Step 3**: Open your file of incoming barcodes.

In [8]:
initialFile = open('barcodes.csv', newline='', encoding='utf-8-sig')
barcode_file = csv.reader(initialFile, dialect='excel')

**Step 4**: Construct and send the data payload to FOLIO for each reserve item.

In [9]:
for each in barcode_file:
    payload = {}
    copiedItem = {}
    payload['courseListingId'] = courseListingIdForLoad
    copiedItem['barcode'] = each[0]
    payload['copiedItem'] = copiedItem
    requestPayload = json.dumps(payload)
    sendReserve = requests.post(uploadReserveUrl, headers=postHeaders, data=requestPayload)
    print(sendReserve.text)

{
  "id" : "798296d4-91a1-4073-ac9c-89e10d8ed58d",
  "courseListingId" : "cef52efb-b3fd-4450-9960-1745026a99d1",
  "itemId" : "d6f7c1ba-a237-465e-94ed-f37e91bc64bd",
  "copiedItem" : {
    "barcode" : "4539876054383",
    "temporaryLocationId" : "53cf956f-c1df-410b-8bea-27f712cca7c0",
    "permanentLocationId" : "53cf956f-c1df-410b-8bea-27f712cca7c0",
    "title" : "Bridget Jones's Baby: the diaries",
    "contributors" : [ {
      "name" : "Fielding, Helen",
      "contributorNameTypeId" : "2b94c631-fca9-4892-a730-03ee529ffe2a"
    } ],
    "publication" : [ {
      "publisher" : "Alfred A. Knopf",
      "place" : "New York",
      "dateOfPublication" : "2016"
    } ],
    "callNumber" : "PR6056.I4588 B749 2016",
    "copy" : "Copy 3",
    "instanceId" : "7fbd5d84-62d1-44c6-9c45-6cb173998bbd",
    "instanceHrid" : "inst000000000006",
    "instanceDiscoverySuppress" : true,
    "holdingsId" : "fb7b70f1-b898-4924-a991-0e4b6312bb5f"
  },
  "temporaryLoanTypeId" : "e8b311a6-3b21-43f2-a269